# Tanzania CEFA 2016 Nexus EE dataset
This notebook is used to create the dataset for CEFA 2016 nexus EE. The dataset consists of multiple files.
- Q_nexus_HH_Analysis_General
- Q_nexus_HH_Analysis_Supply
- Q_nexus_HH_Analysis_Appliances
- Q_nexus_US_Analysis_General
- Q_nexus_US_Analysis_Supply
- Q_nexus_US_Analysis_Appliances
- Village_GPS_info
- GADM_level_3
- GADM_level_1
- GADM_level_2




In [55]:
import os
import sys
import pandas as pd
import numpy as np

sys.path.append("../../")  # Adds higher directory to python modules path.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
from core.ODEDataset import ODEDataset
from utils import common_modifiers, appliances_modifiers, geospatial, socio_modifiers

import pathlib
from utils import constants


In [56]:
DB_ROOT = "../../playground/data/CEFA_Tanzania/2016_EE_nexus"

root = pathlib.Path(DB_ROOT)

ID_COL = "ID"

## Clusters
Clusters are group of columns used together to create a new feature. 


## Categories
Map the original values to the standard values.


In [57]:
Education_level_original2final = {
    'N': 'No schooling',
    'P': 'Primary education',
    'S': 'Secondary education',
}

Socio_status_original2final = {
    'Agr': 'Own-account worker farm',
    'Self': 'Own-account worker non-farm',
    'Agr; Self': 'Own-account worker farm',
}

Connection_type_original2final = {
    'Grid': 'National grid',
    'Grid GenSet': 'National grid',
    'Grid Solar': 'National grid',
    'Solar': 'Solar Home System',
}

Tariff_payment_frequency_original2final = {
    'W': 'Weekly',
    'W2': 'Every 2 weeks',
    'M/2': 'Every 2 weeks',
    'M': 'Monthly',
    'D2': 'Other',
    "Don't know": 'Other',
}


## General
Merge the general information from the datasets:
- Q_nexus_HH_Analysis_General
- Q_nexus_US_Analysis_General

**The general information includes**:
- Monthly_expenditure
- Number_of_rooms
- Number_adults
- Education_level_HHH
- Age_HHH
- Socio_status_HHH
- Climate_zone_lev_1
- Climate_zone_lev_2

In [58]:
Q_nexus_HH = ODEDataset("Q_nexus_HH")
Q_nexus_US = ODEDataset("Q_nexus_US")

Q_nexus_HH.from_excel(root.joinpath("Q_nexus_HH_Analysis_General.xlsx"))
Q_nexus_US.from_excel(root.joinpath("Q_nexus_US_Analysis_General.xlsx"))




### GeoSpatial Information

In [59]:
Q_nexus_HH = Q_nexus_HH.apply(common_modifiers.rename({
    "Q_nexus_HH": ID_COL,
}))

Q_nexus_US = Q_nexus_US.apply(common_modifiers.rename({
    "US_size": 'Size',
    'Q_nexus_US': ID_COL,
}))

hh_cols = Q_nexus_HH.get_columns()
us_cols = Q_nexus_US.get_columns()

cols_in_hh_not_in_us = [c for c in hh_cols if c not in us_cols]
cols_in_us_not_in_hh = [c for c in us_cols if c not in hh_cols]

Q_nexus_HH = Q_nexus_HH.apply(common_modifiers.add_const_driver_many({
    c: np.nan for c in cols_in_us_not_in_hh
}))

Q_nexus_US = Q_nexus_US.apply(common_modifiers.add_const_driver_many({
    c: np.nan for c in cols_in_hh_not_in_us
}))

Q_General = Q_nexus_HH.concat(Q_nexus_US)
# Drop unnecessary columns
Q_General = Q_General.drop_columns(['Comments', 'Changes', 'Issues', 'Impact', 'Connection_reason'])
Q_General = Q_General.group_by(ID_COL)
Q_General.preview(300)


,ID,Village,Interviewee,Role,Age,Sex,Education,Members,Rooms,Size,...,Competition,Customers,Sales_min,Sales_max,Expenses,Problems,Owner_other,Connection,Impact_other,Impact_spec
0,Q_nexus_HH_01,[Nyombo],[Esta],[Mama],[60],[F],[P],[6.0],[7.0],[60],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
1,Q_nexus_HH_02,[Nyombo],[Bernard],[Head],[28],[M],[P],[4.0],[5.0],[40],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
2,Q_nexus_HH_03,[Nyombo],[Rahel],[Mama],[35],[F],[P],[6.0],[6.0],[40],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
3,Q_nexus_HH_04,[Nyombo],[Elina],[Mama],[51],[F],[P],[4.0],[5.0],[40],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
4,Q_nexus_HH_05,[Nyombo],[Sela],[Mama],[28],[F],[P],[4.0],[6.0],[27],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
5,Q_nexus_HH_06,[Nyombo],[Wilbert],[Baba],[24],[M],[P],[2.0],[4.0],[25],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
6,Q_nexus_HH_07,[Nyombo],[Iusta],[Mama],[39],[F],[P],[7.0],[13.0],[65],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
7,Q_nexus_HH_08,[Nyombo],[Flora],[Daughter],[22],[F],[P],[4.0],[7.0],[35],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
8,Q_nexus_HH_09,[Nyombo],[Natila],[Mama],[45],[F],[N],[7.0],[8.0],[150],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
9,Q_nexus_HH_10,[Nyombo],[France Kaduma],[Head],[52],[M],[P],[7.0],[10.0],[70],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]


In [60]:
gadm_level_1_df = pd.read_excel(root.joinpath("GADM_level_1.xlsx"))
gadm_level_2_df = pd.read_excel(root.joinpath("GADM_level_2.xlsx"))
gadm_level_3_df = pd.read_excel(root.joinpath("GADM_level_3.xlsx"))

village_gps_info_df = pd.read_excel(root.joinpath("Village_GPS_info.xlsx"))

In [61]:
Q_General = Q_General.apply(common_modifiers.add_const_driver('GADM_level_0', 'Tanzania'))
Q_General = Q_General.new_feature("Township/Village",
                                  lambda row: row["Village"])

Q_General = Q_General.new_feature("GADM_level_1",
                                  geospatial.gis_info_by_village_level("GADM_level_1",
                                                                       village_gps_info_df))

Q_General = Q_General.new_feature("GADM_level_2",
                                  geospatial.gis_info_by_village_level("GADM_level_2",
                                                                       village_gps_info_df))

Q_General = Q_General.new_feature("GADM_level_3",
                                  geospatial.gis_info_by_village_level("GADM_level_3",
                                                                       village_gps_info_df))

Q_General = Q_General.new_feature("Climate_zone_lev_1",
                                  geospatial.gis_info_by_gadm_level('Climate_majority',
                                                                    gadm_level_1_df,
                                                                    'GADM_level_1'))
Q_General = Q_General.new_feature("Climate_zone_lev_2",
                                  geospatial.gis_info_by_gadm_level('Climate_majority',
                                                                    gadm_level_2_df,
                                                                    'GADM_level_2'))


In [62]:
Q_General.preview()

,ID,Village,Interviewee,Role,Age,Sex,Education,Members,Rooms,Size,...,Connection,Impact_other,Impact_spec,GADM_level_0,Township/Village,GADM_level_1,GADM_level_2,GADM_level_3,Climate_zone_lev_1,Climate_zone_lev_2
0,Q_nexus_HH_01,[Nyombo],[Esta],[Mama],[60],[F],[P],[6.0],[7.0],[60],...,[nan],[nan],[nan],Tanzania,[Nyombo],Njombe,Njombe,Ikuna,12,12
1,Q_nexus_HH_02,[Nyombo],[Bernard],[Head],[28],[M],[P],[4.0],[5.0],[40],...,[nan],[nan],[nan],Tanzania,[Nyombo],Njombe,Njombe,Ikuna,12,12
2,Q_nexus_HH_03,[Nyombo],[Rahel],[Mama],[35],[F],[P],[6.0],[6.0],[40],...,[nan],[nan],[nan],Tanzania,[Nyombo],Njombe,Njombe,Ikuna,12,12
3,Q_nexus_HH_04,[Nyombo],[Elina],[Mama],[51],[F],[P],[4.0],[5.0],[40],...,[nan],[nan],[nan],Tanzania,[Nyombo],Njombe,Njombe,Ikuna,12,12
4,Q_nexus_HH_05,[Nyombo],[Sela],[Mama],[28],[F],[P],[4.0],[6.0],[27],...,[nan],[nan],[nan],Tanzania,[Nyombo],Njombe,Njombe,Ikuna,12,12


In [63]:
Q_General = Q_General.apply(common_modifiers.rename({
    'Expenditures': 'Monthly_expenditure',
    'Rooms': 'Number_of_rooms',
})).apply(
    common_modifiers.take('Monthly_expenditure', 0)).apply(
    common_modifiers.take('Number_of_rooms', 0))

Q_General = Q_General.new_feature("Number_adults", lambda x: x['Members'][0] * 0.535)
Q_General = Q_General.new_feature("Education_level_HHH",
                                  common_modifiers.categorize("Education", Education_level_original2final)).apply(
    common_modifiers.take('Education_level_HHH', 0))
Q_General = Q_General.new_feature("Socio_status_HHH",
                                  common_modifiers.categorize("Income source", Socio_status_original2final)).apply(
    common_modifiers.take('Socio_status_HHH', 0))
# Age_HHH

Q_General = Q_General.new_feature("Age_HHH", socio_modifiers.extract_age_of_head("Age", "Role", "Head"))
Q_General.preview()

,ID,Village,Interviewee,Role,Age,Sex,Education,Members,Number_of_rooms,Size,...,Township/Village,GADM_level_1,GADM_level_2,GADM_level_3,Climate_zone_lev_1,Climate_zone_lev_2,Number_adults,Education_level_HHH,Socio_status_HHH,Age_HHH
0,Q_nexus_HH_01,[Nyombo],[Esta],[Mama],[60],[F],[P],[6.0],7.0,[60],...,[Nyombo],Njombe,Njombe,Ikuna,12,12,3.21,Primary education,Own-account worker farm,NaN
1,Q_nexus_HH_02,[Nyombo],[Bernard],[Head],[28],[M],[P],[4.0],5.0,[40],...,[Nyombo],Njombe,Njombe,Ikuna,12,12,2.14,Primary education,Own-account worker farm,28.0
2,Q_nexus_HH_03,[Nyombo],[Rahel],[Mama],[35],[F],[P],[6.0],6.0,[40],...,[Nyombo],Njombe,Njombe,Ikuna,12,12,3.21,Primary education,Own-account worker farm,NaN
3,Q_nexus_HH_04,[Nyombo],[Elina],[Mama],[51],[F],[P],[4.0],5.0,[40],...,[Nyombo],Njombe,Njombe,Ikuna,12,12,2.14,Primary education,Own-account worker farm,NaN
4,Q_nexus_HH_05,[Nyombo],[Sela],[Mama],[28],[F],[P],[4.0],6.0,[27],...,[Nyombo],Njombe,Njombe,Ikuna,12,12,2.14,Primary education,Own-account worker farm,NaN


In [64]:
Q_General = Q_General.select(
    [ID_COL,
     'Monthly_expenditure', 'Number_of_rooms', 'Number_adults',
     'Education_level_HHH', 'Age_HHH', 'Socio_status_HHH',
     'Climate_zone_lev_1', 'Climate_zone_lev_2',

     ])
Q_General.preview()

,ID,Monthly_expenditure,Number_of_rooms,Number_adults,Education_level_HHH,Age_HHH,Socio_status_HHH,Climate_zone_lev_1,Climate_zone_lev_2
0,Q_nexus_HH_01,30000.0,7.0,3.21,Primary education,NaN,Own-account worker farm,12,12
1,Q_nexus_HH_02,30000.0,5.0,2.14,Primary education,28.0,Own-account worker farm,12,12
2,Q_nexus_HH_03,15000.0,6.0,3.21,Primary education,NaN,Own-account worker farm,12,12
3,Q_nexus_HH_04,20000.0,5.0,2.14,Primary education,NaN,Own-account worker farm,12,12
4,Q_nexus_HH_05,15000.0,6.0,2.14,Primary education,NaN,Own-account worker farm,12,12


## Supply
Q_nexus_HH_Analysis_Supply + Q_nexus_US_Analysis_Supply
- Measurement_age
- Tariff_payment_frequency


In [65]:
Q_nexus_HH_supply = ODEDataset("Q_nexus_HH_supply")
Q_nexus_US_supply = ODEDataset("Q_nexus_US_supply")

Q_nexus_HH_supply.from_excel(root.joinpath("Q_nexus_HH_Analysis_Supply.xlsx"))
Q_nexus_US_supply.from_excel(root.joinpath("Q_nexus_US_Analysis_Supply.xlsx"))


In [66]:
Q_nexus_HH_supply = Q_nexus_HH_supply.apply(common_modifiers.rename({
    "Q_nexus_HH": ID_COL,
}))
Q_nexus_US_supply = Q_nexus_US_supply.apply(common_modifiers.rename({
    'Q_nexus_US': ID_COL,
})).drop_columns(['Acces_date_old', 'EE_importance'])
Q_Supply = Q_nexus_HH_supply.concat(Q_nexus_US_supply)

Q_Supply.preview(300)


,ID,Connection,Acces_date,Metering,Bill_time,Bill_cost,Blackouts,Damages,Damages_quant,Supply_comments
0,Q_nexus_HH_04,Solar,2012.0,Shared,NaN,NaN,0,N,No,No
1,Q_nexus_HH_06,Solar,2015.0,Indiv,NaN,NaN,0,N,No,Far line
2,Q_nexus_HH_07,Solar,2012.0,Indiv,NaN,NaN,0,N,No,No
3,Q_nexus_HH_09,Grid,2013.0,Indiv,M,10000.0,3,N,No,Expensive
4,Q_nexus_HH_10,Grid,2009.0,Indiv,M,15000.0,2,Y,Pump,Expensive
5,Q_nexus_HH_11,Grid,2008.0,Indiv,M/2,NaN,3,N,No,Frequent cut-offs; buy 20 units
6,Q_nexus_HH_12,Grid,2009.0,Shared,M,20000.0,3,Y,Phones,Frequent cut-offs
0,Q_nexus_US_01,Grid,2013.0,Shared,M,4000,2,Y,Changed 4 bulbs since started,Frequent cut-offs; Insufficient power supply;...
1,Q_nexus_US_03,Grid,2016.0,Shared,M,400,3,N,No,No
2,Q_nexus_US_04,Grid,2016.0,Shared,W2,2000,2,N,No,Frequent cut-offs


In [67]:

Survey_date = 2016
Q_Supply = Q_Supply.new_feature("Measurement_age", lambda x: Survey_date - x['Acces_date'])
Q_Supply = Q_Supply.new_feature("Tariff_payment_frequency",
                                common_modifiers.categorize("Bill_time", Tariff_payment_frequency_original2final))

Q_Supply = Q_Supply.select([ID_COL, "Measurement_age", "Tariff_payment_frequency"])

In [68]:
Q_Supply.preview()

,ID,Measurement_age,Tariff_payment_frequency
0,Q_nexus_HH_04,4.0,NaN
1,Q_nexus_HH_06,1.0,NaN
2,Q_nexus_HH_07,4.0,NaN
3,Q_nexus_HH_09,3.0,Monthly
4,Q_nexus_HH_10,7.0,Monthly


## Appliances
General Appliances sections
- Presence_phone_charger
- Presence_iron
- Presence_TV
- Presence_refrigerator/freezer
- Presence_radio/stereo
- Presence_DVD_player
- Presence_fan


In [69]:

US_Appliances = ODEDataset("Q_nexus_US_Analysis_Appliances.xlsx")
HH_Appliances = ODEDataset("Q_nexus_HH_Analysis_Appliances.xlsx")

US_Appliances.from_excel(root.joinpath("Q_nexus_US_Analysis_Appliances.xlsx"))
HH_Appliances.from_excel(root.joinpath("Q_nexus_HH_Analysis_Appliances.xlsx"))

US_Appliances = US_Appliances.apply(common_modifiers.rename({
    "Q_nexus_US": ID_COL,
}))

HH_Appliances = HH_Appliances.apply(common_modifiers.rename({
    "Q_nexus_HH": ID_COL,
}))

Q_Appliances = HH_Appliances.concat(US_Appliances).group_by(ID_COL)
Q_Appliances.preview()



,ID,Item,Item_power,Item_number,Time_average,Time_minimum,ST_FW1,ET_FW1,ST_FW2,ET_FW2,Date_num_1,Date_num_2,Date_num_3,Unnamed: 13,Unnamed: 14
0,Q_nexus_HH_04,"[Ind_lights, Out_lights]","[18, 18]","[1, 1]","[720, 720]","[720, 720]","[0, 0]","[420, 420]","[1140, 1140]","[1440, 1440]","[2012 1, 2012 1]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
1,Q_nexus_HH_06,"[Ind_lights, Radio]","[18, 10]","[1, 1]","[660, 420]","[660, 180]","[0, 360]","[360, 1320]","[1140, 0]","[1440, 0]","[2015 1, 2015 1]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, nan]"
2,Q_nexus_HH_07,"[Charger, Decoder, Ind_lights, Out_lights, TV]","[5, 5, 18, 18, 80]","[4, 1, 11, 2, 1]","[120, 240, 720, 720, 240]","[120, 240, 720, 720, 240]","[720, 720, 0, 0, 720]","[1200, 1260, 360, 360, 1260]","[0, 0, 1080, 1080, 0]","[0, 0, 1440, 1440, 0]","[Before connection, 2012 1, 2012 11, 2012 2, 2...","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
3,Q_nexus_HH_09,"[Charger, Ind_lights, Out_lights, Radio, Speak...","[5, 18, 18, 10, 40, 80]","[4, 10, 4, 2, 1, 1]","[150, 180, 180, 180, 120, 120]","[60, 60, 60, 180, 120, 60]","[0, 1080, 1080, 1020, 1080, 1140]","[1440, 1260, 1260, 1320, 1260, 1260]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[Before connection, 2013 10, 2013 4, 2013 1, 2...","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]"
4,Q_nexus_HH_10,"[Charger, Ind_lights, Out_lights, Pump, Radio,...","[5, 18, 18, 300, 10, 80]","[2, 10, 2, 1, 2, 1]","[480, 180, 660, 30, 420, 180]","[60, 60, 660, 30, 420, 180]","[0, 1140, 0, 1080, 360, 1140]","[360, 1260, 360, 1320, 480, 1260]","[1320, 0, 1140, 0, 1080, 0]","[1440, 0, 1440, 0, 1260, 0]","[Before connection, 2009 10, 2009 2, 2011 1, 2...","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]"


In [70]:
Q_Appliances = Q_Appliances.new_feature("Presence_phone_charger",
                                            appliances_modifiers.presence_appliances("Item", "Charger"))
Q_Appliances = Q_Appliances.new_feature("Presence_iron", appliances_modifiers.presence_appliances("Item", "Iron"))

Q_Appliances = Q_Appliances.new_feature("Presence_TV", appliances_modifiers.presence_appliances("Item", "TV"))

Q_Appliances = Q_Appliances.new_feature("Presence_refrigerator/freezer",
                                        appliances_modifiers.presence_appliances("Item", "Refrigerator"))

Q_Appliances = Q_Appliances.new_feature("Presence_stereo", appliances_modifiers.presence_appliances("Item", "Stereo"))
Q_Appliances = Q_Appliances.new_feature("Presence_radio", appliances_modifiers.presence_appliances("Item", "Radio"))
Q_Appliances = Q_Appliances.new_feature("Presence_radio/stereo",
                                        common_modifiers.multi_unify_presence(
                                            ['Presence_radio', 'Presence_stereo']))

Q_Appliances = Q_Appliances.new_feature("Presence_fan", appliances_modifiers.presence_appliances("Item", "Fan"))
Q_Appliances = Q_Appliances.new_feature("Presence_DVD_player", lambda x: 0)

Q_Appliances = Q_Appliances.select([
    ID_COL, "Presence_phone_charger", "Presence_iron", "Presence_TV", "Presence_refrigerator/freezer",
    "Presence_radio/stereo", "Presence_DVD_player", "Presence_fan"
])

Q_Appliances.preview(1000)



,ID,Presence_phone_charger,Presence_iron,Presence_TV,Presence_refrigerator/freezer,Presence_radio/stereo,Presence_DVD_player,Presence_fan
0,Q_nexus_HH_04,0,0,0,0,0,0,0
1,Q_nexus_HH_06,0,0,0,0,1,0,0
2,Q_nexus_HH_07,1,0,1,0,0,0,0
3,Q_nexus_HH_09,1,0,1,0,1,0,0
4,Q_nexus_HH_10,1,0,1,0,1,0,0
5,Q_nexus_HH_11,1,0,1,0,1,0,0
6,Q_nexus_HH_12,1,0,1,0,1,0,0
7,Q_nexus_US_01,0,0,0,0,0,0,0
8,Q_nexus_US_03,1,0,0,0,0,0,1
9,Q_nexus_US_04,1,0,0,0,0,0,0


In [71]:
CEFA_2016_EE = Q_General.merge(Q_Appliances, ID_COL)
CEFA_2016_EE = CEFA_2016_EE.merge(Q_Supply, ID_COL)

CEFA_2016_EE = CEFA_2016_EE.apply(common_modifiers.rename({
    ID_COL: "ID"
}))
CEFA_2016_EE = CEFA_2016_EE.apply(common_modifiers.add_const_driver_many({
    "Hours_available_electricity": np.nan,
    "Years_of_HHH_in_community": np.nan,
    'Dwelling_quality_index': np.nan,
    'Ownership_motorized_vehicle': np.nan,
    'Ownership_small_livestock': np.nan,
    'Clean_fuel': np.nan,
    'Ownership_large_livestock': np.nan,
    'HH_with_home_business': np.nan,
}))
CEFA_2016_EE = CEFA_2016_EE.select(["ID"] + constants.DRIVERS_LIST + constants.PRESENCE_LIST)
CEFA_2016_EE.preview()

,ID,Years_of_HHH_in_community,Dwelling_quality_index,Hours_available_electricity,Measurement_age,Monthly_expenditure,Number_of_rooms,Climate_zone_lev_1,Climate_zone_lev_2,Age_HHH,...,Ownership_large_livestock,Clean_fuel,Tariff_payment_frequency,Presence_refrigerator/freezer,Presence_iron,Presence_fan,Presence_DVD_player,Presence_radio/stereo,Presence_phone_charger,Presence_TV
0,Q_nexus_HH_04,NaN,NaN,NaN,4.0,20000.0,5.0,12,12,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,Q_nexus_HH_06,NaN,NaN,NaN,1.0,30000.0,4.0,12,12,NaN,...,NaN,NaN,NaN,0,0,0,0,1,0,0
2,Q_nexus_HH_07,NaN,NaN,NaN,4.0,60000.0,13.0,12,12,NaN,...,NaN,NaN,NaN,0,0,0,0,0,1,1
3,Q_nexus_HH_09,NaN,NaN,NaN,3.0,50000.0,8.0,12,12,NaN,...,NaN,NaN,Monthly,0,0,0,0,1,1,1
4,Q_nexus_HH_10,NaN,NaN,NaN,7.0,500000.0,10.0,12,12,52.0,...,NaN,NaN,Monthly,0,0,0,0,1,1,1


In [73]:
CEFA_2016_EE.to_csv(root.joinpath("CEFA_2016_nexus_EE.csv"))